In [1]:
import os
from io import open

# set directory for text input files
text_files = os.path.join(os.path.dirname("__file__"), 'texts')
files = os.listdir(text_files)

In [2]:
texts = []
authors = []

# Loop through each text file and append relevant data to lists
for doc in os.listdir(text_files):

    text = open(text_files + '\\' + doc, 'r', encoding='utf-8')
    # open the file, store as lower case
    text = text.read().lower()
    texts.append(text)
    
    authors.append(doc)



In [3]:
n = 10  #ngram size

In [4]:
n_grams = set([])
treated_texts = []

import regex as re
import string

for text in texts:
    
    # Remove high frequency stop words, as in Sims/Wu et. al
    stopwords = ['and', 'the', 'a', 'an']
    #text = text.split()
    
    # removes all punctuation, spaces and new lines    
    #text = re.sub(ur"\p{P}+", "", text)
    
    # Split text, remove stopwords and rejoin string without spaces or newlines
    text = text.split()

    text = [x for x in text if x not in stopwords]
    
    text = ''.join(text)   
    
    #produces 6 character n-grams in a 'sliding window' across the text
    text = [text[i:i+n] for i in range(0, len(text), 1) if not any(j in string.punctuation for j in text[i:i+n])]
    treated_texts.append(text)   
    #n_gram_frequency = Counter(text)
    
    # Append features to the ngram list
    from collections import Counter
    c = Counter(text)
    
   

    # Append features to the ngram list
    for key, val in c.iteritems():
        #if not any(i in string.punctuation for i in key)
        n_grams.add(key)
        
           
                
             

In [5]:
print len(n_grams)

913408


In [6]:
from collections import OrderedDict

n_grams = OrderedDict.fromkeys(n_grams, 0.0) 

frequencies = []

t = []

for text in treated_texts:
    c = Counter(text)
    temp = OrderedDict(n_grams)
    
    for key, val in c.iteritems():
        if len(key) == n:
            temp[key] = float(val)

    frequencies.append(temp.values()) 

In [7]:
from scipy.stats import entropy
from numpy.linalg import norm
import numpy as np

def JSD(P, Q):
    P = np.array(P, dtype = 'float_')
    Q = np.array(Q, dtype = 'float_')
    P = P / np.sum(P)
    Q = Q / np.sum(Q)
    M = (P + Q) / 2
    jsd = 0.5 * (entropy(P, M) + entropy(Q, M))
    return jsd

In [8]:
#TODO - make this more efficient

distance_matrix = [[] for x in range(len(frequencies))]

for i in range(len(frequencies)):
    for j in range(len(frequencies)):
        
        if i==j:
            distance_matrix[i].append(0)
            break
        
        distance_matrix[i].append(JSD(frequencies[i], frequencies[j]))  

In [9]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo.TreeConstruction import _DistanceMatrix 
from Bio import Phylo
from pygraphviz import *


m = _DistanceMatrix(authors, distance_matrix)


constructor = DistanceTreeConstructor()

njtree = constructor.nj(m)
Phylo.write(njtree, '10', 'newick')
# njtree = Phylo.read('treefile', 'newick')
Phylo.draw_ascii(njtree)



  _________________________________________________ wilson 1.txt
 |
 |__________________________________________________ fergusson 1.txt
 |
 |__________________________________________________ hogg 3.txt
_|
 | _________________________________________________ hogg 4.txt
 ||
 ||_________________________________________________ hogg 2.txt
 ||
 ||_________________________________________________ hogg 1.txt
 |
 |__________________________________________________ tannahill 1.txt
 |
 | __________________________________________________ Burns4.txt
 ||
 ,|             _____________________________________ Burns3.txt
 || ___________|
 |||           |____________________________________ Burns2.txt
 | |
 | |_________________________________________________ 10randomBurnspoems.txt
 |
 |____________________________________________________ B-List 1.txt
 |
 |___________________________________________________ A-List 1.txt
 |
 |__________________________________________________ Burns1.txt



In [10]:
# # PCA using the ngram frequencies, just for laughs

# import numpy as np

# ###### Convert both X and Y to numpy arrays
# X = np.array(frequencies)
# y = np.array(authors)
# markers = set(authors)

# # Standardise data, perform PCA
# from sklearn.decomposition import PCA as sklearnPCA
# from sklearn.preprocessing import StandardScaler

# X_std = StandardScaler().fit_transform(X)
# sklearn_pca = sklearnPCA(n_components=2)
# Y_sklearn = sklearn_pca.fit_transform(X_std)

# import plotly.plotly as py
# from plotly.graph_objs import *
# import plotly.tools as tls

# traces = []

# for author in markers:

#     trace = Scatter(
#         x=Y_sklearn[y==author,0],
#         y=Y_sklearn[y==author,1],
#         mode='markers',
#         name=author,
#         marker=Marker(
#             size=14))
#     traces.append(trace)


# data = Data(traces)
# layout = Layout(title='PCA of Burns and contemporaries',
#                 xaxis=XAxis(title='Principal Component 1'),
#                 yaxis=YAxis(title='Principal Component 2'))
# fig = Figure(data=data, layout=layout)
# py.iplot(fig)

In [11]:
# from ete3 import Tree, TreeStyle
# t = Tree('newck', format=1)
# ts.show_leaf_name = True
# ts.scale =  120 # 120 pixels per branch length unit
# t.show(tree_style=ts)